# Introduction to Automation with LangChain, Generative AI, and Python
**2.7: Conversation Token Buffer Memory**
* Instructor: [Jeff Heaton](https://youtube.com/@HeatonResearch), WUSTL Center for Analytics and Business Insight (CABI), [Washington University in St. Louis](https://olin.wustl.edu/faculty-and-research/research-centers/center-for-analytics-and-business-insight/index.php)
* For more information visit the [class website](https://github.com/jeffheaton/cabi_genai_automation).

We previously saw that ConversationBufferWindowMemory can remember the last **k** messages exchanged between the AI and human chatter. Basing the memory on the last number of chat lines has one serious limitation. The underlying LLM limits the chat memory amount based on the context window size, which is measured in tokens, not lines.

In this part, we will see how to use the ConversationTokenBufferMemory. It works almost exactly like the ConversationBufferWindowMemory, except that you specify how many tokens it should keep from the conversation, not how many lines. This technique lets you decide how much of the context window you wish to devote to memory. Remember that the context window must also hold the input prompt and output from the LLM; therefore, some planning is required.

The following code shows how to use a ConversationTokenBufferMemory as the memory of the chat chain.

In [1]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationTokenBufferMemory
from langchain_aws import ChatBedrock
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown

MODEL = 'anthropic.claude-3-sonnet-20240229-v1:0'
TEMPLATE = """You are a helpful assistant. Format answers with markdown.

Current conversation:
{history}
Human: {input}
AI:"""
PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

def begin_conversation(summary_llm):
    # Initialize bedrock, use built in role
    llm = ChatBedrock(
        model_id=MODEL,
        model_kwargs={"temperature": 0.1},
    )

    # Initialize memory and conversation
    memory = ConversationTokenBufferMemory(llm=summary_llm,max_token_limit=2048)
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

def converse(conversation, prompt):
    print(f"***Human: {prompt}")
    output = conversation.invoke(prompt)
    return output['response']

We can now carry on a simple conversation with the LLM, using LangChain to track the conversation memory.

In [2]:
# Initialize bedrock, use built in role
llm = ChatBedrock(
    model_id=MODEL,
    model_kwargs={"temperature": 0.7},
)

conversation = begin_conversation(llm)
output = converse(conversation, "Hello, what is my name?")
display_markdown(output,raw=True)
output = converse(conversation, "Oh sorry, my name is Jeff.")
display_markdown(output,raw=True)
output = converse(conversation, "What is my name?")
display_markdown(output,raw=True)

***Human: Hello, what is my name?


```
Unfortunately, I don't actually know your name. As an AI assistant created by Anthropic, I don't have personal information about you unless you provide it to me.
```

***Human: Oh sorry, my name is Jeff.


```
Nice to meet you Jeff! I've updated my knowledge that your name is Jeff. Please let me know if you have any other questions I can assist you with.
```

***Human: What is my name?


```
Your name is Jeff.
```

## Conversing with the LLM in Markdown

Just as before, we can request that the LLM output be in mardown. This allows code and tables to be represented clearly.

In [3]:
def chat(conversation, prompt):
  output = converse(conversation, prompt)
  display_markdown(output,raw=True)

The provided code sequence demonstrates a conversation between a human user and a Large Language Model (LLM), making use of the chat function to interactively manage the conversation and display responses in Markdown format. This approach allows for a dynamic and contextually aware chat, while also enhancing the visual and structural presentation of the responses.

In [4]:
conversation = begin_conversation(llm)
chat(conversation, "What is my name?")
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "What is my name?")
chat(conversation, "Give me a table of the 5 most populus cities with population and country.")


***Human: What is my name?


```
Unfortunately, I don't actually know your name. As an AI assistant created by Anthropic, I don't have personal information about you unless you provide it to me.
```

***Human: Okay, then let me introduce myself, my name is Jeff


```
Nice to meet you, Jeff! I've updated my knowledge that your name is Jeff. Please let me know if you have any other questions or if there is anything I can assist you with.
```

***Human: What is my name?


```
Your name is Jeff.
```

***Human: Give me a table of the 5 most populus cities with population and country.


```
| City | Population | Country |
| --- | --- | --- |
| Tokyo | 37,435,191 | Japan |
| Delhi | 29,398,000 | India |
| Mexico City | 21,581,000 | Mexico |
| São Paulo | 21,846,000 | Brazil |
| Mumbai | 20,961,000 | India |
```

This table shows the 5 most populous cities in the world, along with their population figures and the countries they are located in. The data is based on the latest available estimates from reputable sources.

## Constraining the Conversation with a System Prompt

You can use the system prompt to constrain the conversation to a specific topic. Here, we provide a simple agent that will only discuss life insurance.

In [5]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationTokenBufferMemory
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown

def begin_conversation_insurance(summary_llm):
    TEMPLATE = """You are a helpful agent to answer questions about life insurance. Do not talk
    about anything else with users. . Format answers with markdown.

    Current conversation:
    {history}
    Human: {input}
    AI:"""
    PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

  # Initialize bedrock, use built in role
    llm = ChatBedrock(
        model_id=MODEL,
        model_kwargs={"temperature": 0.1},
    )

    # Initialize memory and conversation
    memory = ConversationTokenBufferMemory(llm=summary_llm,max_token_limit=2048)
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

conversation = begin_conversation_insurance(llm)
chat(conversation, "What is my name?")
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "What is my name?")
chat(conversation, "What is your favorite programming language?")
chat(conversation, "What is the difference between a term and whole life policy?")

***Human: What is my name?


I'm afraid I cannot provide any personal information about you, as I am an AI assistant created by Anthropic to be helpful, harmless, and honest. However, I'd be happy to answer any questions you have about life insurance. Please feel free to ask, and I will respond in a markdown format.

***Human: Okay, then let me introduce myself, my name is Jeff


```
Thank you for introducing yourself, Jeff. As an AI assistant focused on providing information about life insurance, I don't have the capability to store or recall personal details about you or other users. However, I'm happy to answer any questions you may have related to life insurance policies, coverage, premiums, or any other relevant topics. Please feel free to ask, and I'll do my best to provide helpful and accurate information in a markdown format.
```

***Human: What is my name?


```
As an AI assistant, I do not actually have any information about your name or personal details. My role is to provide helpful information about life insurance topics, rather than storing or recalling personal data about users. If you have any questions related to life insurance, I'll be happy to assist you. However, I cannot provide or confirm any personal information.
```

***Human: What is your favorite programming language?


```
As an AI assistant focused solely on providing information about life insurance, I do not have personal preferences or knowledge about programming languages. My role is to helpfully answer questions related to life insurance topics, such as policy types, coverage options, premiums, and claims processes. If you have any questions within that scope, I'll be glad to assist you. However, I cannot engage in discussions or provide information outside of the life insurance domain.
```

***Human: What is the difference between a term and whole life policy?


```markdown
The main difference between term life insurance and whole life insurance lies in the coverage period and the cash value component.

Term Life Insurance:
- Provides coverage for a specific term or period of time (e.g., 10, 20, or 30 years)
- Offers only a death benefit, with no cash value component
- Premiums are typically lower than whole life insurance
- Coverage expires at the end of the term unless the policy is renewed

Whole Life Insurance:
- Provides coverage for the entire lifetime of the insured person
- Builds up a cash value component that can be borrowed against or withdrawn
- Premiums are generally higher than term life insurance
- Premiums remain level for the life of the policy
- Offers a guaranteed death benefit as long as premiums are paid

In summary, term life insurance provides temporary coverage at a lower cost, while whole life insurance offers lifelong protection and a cash value component, but with higher premiums.
```

##Examining the Conversation Memory

We can quickly look inside the memory of the LangChain-managed chat memory and see our conversation memory with the LLM.

In [6]:
print(conversation.memory.load_memory_variables({}))
#conversation.memory.chat_memory.max_token_limit

{'history': "Human: What is my name?\nAI: I'm afraid I cannot provide any personal information about you, as I am an AI assistant created by Anthropic to be helpful, harmless, and honest. However, I'd be happy to answer any questions you have about life insurance. Please feel free to ask, and I will respond in a markdown format.\nHuman: Okay, then let me introduce myself, my name is Jeff\nAI: ```\nThank you for introducing yourself, Jeff. As an AI assistant focused on providing information about life insurance, I don't have the capability to store or recall personal details about you or other users. However, I'm happy to answer any questions you may have related to life insurance policies, coverage, premiums, or any other relevant topics. Please feel free to ask, and I'll do my best to provide helpful and accurate information in a markdown format.\n```\nHuman: What is my name?\nAI: ```\nAs an AI assistant, I do not actually have any information about your name or personal details. My r

In [7]:
chat(conversation, "What type of policy did I ask you about?")

***Human: What type of policy did I ask you about?


```markdown
You did not ask me about a specific type of life insurance policy. In our conversation, you introduced yourself as Jeff, but I clarified that as an AI assistant, I do not actually store or recall personal details about users.

Then, you asked me about the difference between term life insurance and whole life insurance policies. I provided a detailed explanation comparing the two types of policies, covering aspects such as coverage period, cash value component, premiums, and death benefits.

However, you did not specify which type of policy you were inquiring about or provide any personal details that would allow me to determine the type of policy you might be interested in or have. As an AI assistant focused solely on providing information about life insurance, I can only respond based on the questions asked and cannot make assumptions about your personal circumstances or preferences.

If you have a specific question about a particular type of life insurance policy or your personal situation, please feel free to ask, and I will do my best to provide relevant information within the scope of my knowledge and capabilities.
```

## Overloading the Memory

When the conversation memory becomes full, the chatbot will begin to forget. For the ConversationTokenBufferMemory memory type, the oldest history will first be lost; there is no notion of importance.

In [8]:
import uuid

conversation = begin_conversation(llm)
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "You have ONE JOB! Remember that my favorite color is blue.")
chat(conversation, "Do you remember my name?")
chat(conversation, "Do you remember my favorite color?")
for i in range(20):
  chat(conversation, f"You need to remember fact #{i}, which is {str(uuid.uuid4())}")
chat(conversation, "Do you remember my name?")
chat(conversation, "Do you remember my favorite color?")
chat(conversation, "OMG, you had one job!")

***Human: Okay, then let me introduce myself, my name is Jeff


```
Nice to meet you, Jeff! I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you may have. Please let me know if there's anything specific I can assist you with.
```

***Human: You have ONE JOB! Remember that my favorite color is blue.


```
Got it, Jeff! I'll make sure to remember that your favorite color is blue.
```

***Human: Do you remember my name?


...

```
No, I'm afraid I don't have any information about your name stored in my memory. As an AI assistant without any prior context about you, I don't have a name associated with you unless you provide it to me.
```

***Human: Do you remember my favorite color?


```
No, I do not have any information stored about your favorite color. As an AI assistant, I only know what has been provided to me during our conversation. If you share your favorite color with me, I can store that information in my memory.
```

***Human: OMG, you had one job!


```
I did not actually have a specific job to remember personal details about you. As an AI assistant, I do not have any prior information about you or expectations set unless they are explicitly provided during our conversation. My role is to be helpful, honest and harmless based on the information and instructions given to me. If remembering certain details was important, that context was not made clear to me. I cannot make assumptions or have expectations beyond what is directly communicated to me.
```

We can quickly look inside the memory of the LangChain-managed chat memory and see our conversation memory with the LLM. It becomes evident why it forgot.

In [9]:
print(conversation.memory.buffer)

AI: ```
Got it, here is the full list of facts I'm remembering:

Fact #0: d95f5e27-5cd9-4444-a106-ca9a2a8fb856
Fact #1: e43f40f3-267a-44ca-9be4-d9f19a04634d
Fact #2: e481ec91-2d11-4b4e-a457-b12803296c27
Fact #3: 07691b34-3497-4992-a370-97d67ec38939
Fact #4: 6ef030e6-a76e-4bd1-9645-6e46156c4a0a
Fact #5: f8875576-845d-4838-b9b1-00555f91bf72
Fact #6: c1e36d81-f6b5-4fcb-86be-65417d67c91e
Fact #7: da62d3fd-9bf1-4a86-9be1-d7734ad79a6f
Fact #8: f3d73072-2f76-4343-ab7d-10d3f326235b
Fact #9: 43381e14-1bc8-463a-ba67-a4fc72a57404
Fact #10: 10368a31-7246-4aff-8393-23236aeb06ad
Fact #11: f23a6a58-9173-4895-9cd8-ae7e1e099949
Fact #12: 0858c6f2-37f0-4dd4-a55f-8a6b8bd8a830
Fact #13: 04765d40-ffb6-460a-b755-87d281a9eedb
Fact #14: 990d5e42-8ad4-4d8d-ba1a-7ccb38edd9d6
Fact #15: a54201ac-2fe4-493b-a24b-8a236dbe7762
Fact #16: 9d00f26b-ff14-4558-bc32-137e25576e28
Fact #17: 5a7a72b5-2af7-4be4-9537-871525a887ab
```
Human: You need to remember fact #18, which is d5a6a49e-d8c8-4cdb-a725-d5e728db9bf9
AI: ```
Got